### The return of a given financial instrument from the previous day has a statistically significant correlation with the return of at least one specific hourly period in the subsequent 24 hours. This relationship persists whether the correlated hour aligns with the opening or closing of a trading session or at an arbitrary time. The strength and directionality of this relationship can be quantified using the Pearson correlation coefficient and/or be tested for predictive causality using Granger causality tests.

In [80]:
import MetaTrader5 as mt5
from datetime import datetime
import pytz
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
import numpy as np
import warnings
import seaborn as sns
# import coint
warnings.filterwarnings("ignore")
mt5.initialize()
account=51127988
password="Aar2frM7"
server = 'ICMarkets-Demo'

In [24]:
def get_rates(pair1, timeframe, x):
    pair1 = pd.DataFrame(mt5.copy_rates_from_pos(pair1, timeframe, 0, x))
    pair1['time'] = pd.to_datetime(pair1['time'], unit = 's')
    return pair1

In [129]:
hourly_df = get_rates('EURUSD.a', mt5.TIMEFRAME_H1, 24)
hourly_df = hourly_df[['time', 'close']]
daily_df = get_rates('EURUSD.a', mt5.TIMEFRAME_D1, 2)
daily_df = daily_df[['time', 'close']]
df = pd.concat([daily_df, hourly_df], axis = 1, join = 'outer')
# # 1. Drop the first 'time' column.
df.drop(columns=['time'], inplace=True)

df.columns = ['close1', 'close2']
df['d_return'] = df['close1'].pct_change()
# df['d_return'] = df['d_return'].ffill(axis = 1)
df['h_return'] = df['close2'].pct_change()

df[['h_return', 'd_return']].head(5)

,h_return,d_return
0,NaN,NaN
1,0.000311,0.005544
2,-0.000064,0.000000
3,0.000393,0.000000
4,0.000137,0.000000


In [130]:
df.head()

,close1,close2,d_return,h_return
0,1.09492,1.09484,NaN,NaN
1,1.10099,1.09518,0.005544,0.000311
2,NaN,1.09511,0.000000,-0.000064
3,NaN,1.09554,0.000000,0.000393
4,NaN,1.09569,0.000000,0.000137


In [121]:
for i,row in df.iterrows():
    print(row)

close1      1.09492
close2      1.09484
d_return        NaN
h_return        NaN
Name: 0, dtype: float64
close1      1.100990
close2      1.095180
d_return    0.005544
h_return    0.000311
Name: 1, dtype: float64
close1           NaN
close2      1.095110
d_return    0.000000
h_return   -0.000064
Name: 2, dtype: float64
close1           NaN
close2      1.095540
d_return    0.000000
h_return    0.000393
Name: 3, dtype: float64
close1           NaN
close2      1.095690
d_return    0.000000
h_return    0.000137
Name: 4, dtype: float64
close1           NaN
close2      1.095560
d_return    0.000000
h_return   -0.000119
Name: 5, dtype: float64
close1           NaN
close2      1.095420
d_return    0.000000
h_return   -0.000128
Name: 6, dtype: float64
close1           NaN
close2      1.095040
d_return    0.000000
h_return   -0.000347
Name: 7, dtype: float64
close1           NaN
close2      1.095210
d_return    0.000000
h_return    0.000155
Name: 8, dtype: float64
close1           NaN
close2     

In [70]:

# Calculate returns
daily_df['return'] = daily_df['close'].pct_change()
hourly_df['return'] = hourly_df['close'].pct_change()

correlations = []

# For each daily return, correlate with each hourly return within the next 24 hours
for index, row in daily_df[:-1].iterrows():  # Exclude the last day since we might not have 24 hourly data points after it
    day = row['time']
    daily_return = row['return']
    
    # Get the 24 hourly data points following this day
    start_time = pd.to_datetime(day)
    end_time = start_time + pd.Timedelta(hours=24)
    hourly_subset = hourly_df[(hourly_df['time'] > start_time) & (hourly_df['time'] <= end_time)]
    
    if not hourly_subset.empty and not pd.isnull(daily_return):
        hourly_returns = hourly_subset['return'].dropna().tolist()
        daily_returns = [daily_return] * len(hourly_returns)
        
        # Now, we can compute the correlation correctly
        if len(hourly_returns) > 1:  # We need at least two data points to compute correlation
            correlation = np.corrcoef(daily_returns, hourly_returns)[0, 1]
            correlations.append(correlation)

if correlations:
    average_correlation = sum(correlations) / len(correlations)
    print(average_correlation)
else:
    print("No correlations found.")

No correlations found.


### The open hour of each corresponding forex session will correlate with atleast one consistent time period on a lower timeframe before the next session opens. E.g. If Syd opens 15+ pips, a 15min bar 10 bars later will also return postive every time. 